In [ ]:
import json
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
import copy
import os
os.chdir('..')

from performance_forecasting.data_creator import download_statement_data_for_exchanges

In [ ]:
import category_encoders as ce


def hash_encode_gt_features(
        df, columns, concat=True, num_encode_dimensions=32, hashing_method='md5'
):
    hash_encoding = ce.HashingEncoder.hashing_trick(
        df[columns], hashing_method=hashing_method, 
        N=num_encode_dimensions, cols=columns
    )
    
    return (
        pd.concat([df, hash_encoding],axis=1), 
        ["col_"+str(i) for i in range(num_encode_dimensions)]
    )

    

def one_hot_encoding(df, columns, concat=True):
    encoder = OneHotEncoder(drop=None).fit(
        df[columns].values
    )
    encodings = encoder.transform(df[columns]).toarray()
    
    encodings = pd.DataFrame(
        data=encodings,
        columns=['encoded_'+str(n) for n in range(encodings.shape[1])]
    ).astype(np.int32)
    
    if concat:
        return pd.concat(
            [df, encodings],
            axis=1
        )
    else:
        return encodings


def add_scaled_vars(df, scale_columns, write_to_new_df=True, prefix='_scaled'):
    min_maxes = []
    write_df = pd.DataFrame() if write_to_new_df else df
    prefix = '' if write_to_new_df else prefix
    
    scale_columns = df.columns if scale_columns == 'all' else scale_columns
    
    for column in scale_columns:
        max_val, min_val = df[column].max(), df[column].min()
        write_df[column+prefix] = (df[column] - min_val) / (max_val - min_val)
        
        min_maxes.append([max_val, min_val])
        
    return write_df, pd.DataFrame(
        data=np.array(min_maxes).T, columns=scale_columns, index=['max', 'min']
    )


def _inverse_log(var, neg=False):
    a = 1 if neg else -1
    b = -1*a
    return a * np.log(b/var)

def _sqrt(var, neg=False):
    a = -1 if neg else 1
    return a * np.sqrt(a*var)

def math_func_scaling(
    df, 
    scale_columns, 
    prefix='_log_scaled', 
    write_to_new_df=True,
    math_func='log'
):
    if math_func == 'log':
        math_func = _inverse_log
    else:
        math_func = _sqrt

    write_df = pd.DataFrame() if write_to_new_df else df
    prefix = '' if write_to_new_df else prefix
    
    maxes = pd.DataFrame()
    
    for column in scale_columns:
        zero_mask = df[column] == 0
        negative_mask = df[column] < 0
        new_column = column+prefix
        
        write_df[new_column] = math_func(df[column])
        
        write_df.loc[zero_mask, new_column] = 0
        write_df.loc[negative_mask, new_column] = math_func(df[column], neg=True)
        
        maxes[new_column] = write_df[new_column].abs().max()
        write_df[new_column] = write_df[new_column]/ write_df[new_column].abs().max()
        
        
        
    return write_df, maxes



def moving_avg(data, k=0.9):
    ema = data[0]
    vals = [data[0]]
    
    for point in data[1:]:
        ema = ema*k + point*(1-k)
        vals.append(ema)
        
    return vals
        

def add_moving_avgs(df, moving_avgs, varnames):
    for k in moving_avgs:
        for varname in varnames:
            df[str(k).replace('.', '_')+'_ema_'+varname] =\
                df[varname].ewm(span=k).mean() #moving_avg(df[varname], k) #
            
    return df


def plot_data(df, columns, x='date'): 
    plt = px.line(
        df,
        x=x,
        y=columns
    )
    plt.show()

In [ ]:
df = pd.DataFrame({'a': [0, 100, 1000, 10000], 'b': [0, -100, -1000, -10000]})

mod_df, _ = math_func_scaling(
    df, 
    scale_columns=['a', 'b'], 
    prefix='', 
    write_to_new_df=True,
    math_func='sqrt'
)

maxes_1 = pd.DataFrame({'a': [1], 'b': [1]})
maxes_0 = pd.DataFrame({'a': [10000], 'b': [10000]})


for col in df.columns:
    mod_df[col] = mod_df[col] * maxes_1[col].values[0]
    mod_df[col] = np.square(mod_df[col])* maxes_0[col].values[0]
    
mod_df

In [ ]:
period = 'year'
min_statements = 2
max_years=30

download_statements = False 

path_prefix = '' #'D:/'
data_path = f"{path_prefix}data/statements/{period}/"
pkl_path='aggregation/sequences'

exchanges = ['NYSE']#[NASDAQ', 'EURONEXT' ,'NYSE', 'LSE']

prefixes =  {'NASDAQ':'', 'EURONEXT':'', 'NYSE':'', 'LSE':''}

if not os.path.isdir(data_path):
    os.makedirs(data_path)
    
print(prefixes[exchanges[0]])

In [ ]:
if download_statements:
    download_statement_data_for_exchanges(
        exchanges, period, min_statements, data_path, prefix=prefixes[exchanges[0]]
    )

In [ ]:
categorical_cols = [
    'symbol', 'year', 'years_since_public', #date', 
    'sector', 'country', 'industry', 'isEtf',
    'exchange', 'isAdr', 'isFund', 'currency',
    'reportedCurrency', 'isActivelyTrading',
]

deleted_cols = ['reportedCurrency', 'fullTimeEmployees']

growth_cols = None
raw_data_filename = 'raw_data.csv'


moving_avgs = [10, 4] if period == 'year' else [50, 20]

In [ ]:
load = True
raw_data = pd.DataFrame()

growth_cols = None
numeric_cols = None

if not load:
    for path in tqdm(os.listdir(data_path)[1:]):
        df = pd.read_csv(data_path+path, index_col=0)
        
        if growth_cols is None:
            growth_cols = list(df.columns[df.columns.str.contains('growth')])
            numeric_cols = [
                col for col in df.columns if col not in categorical_cols+growth_cols+['date']+deleted_cols
            ]

        if 'date' in df.columns:
            try:
                df['date'] = pd.to_datetime(
                    df['date'], format="%Y-%m-%d"
                )
                df = df.sort_values(by='date', axis=0).reset_index(drop=True)

                df['year'] = df['date'].dt.year
                df['years_since_public'] = df.index

                ysp = list(df.index)

                if max(ysp) < 50:
                    df = df.drop(deleted_cols, axis=1)

                    df = add_moving_avgs(
                        df,
                        moving_avgs,
                        numeric_cols
                    )

                    df['next_year_revenue'] = df['revenue'].shift(-1)
                    df['next_year_freeCashFlow'] = df['freeCashFlow'].shift(-1)

                    raw_data = raw_data.append(df, ignore_index=True)
                else:
                    print(df)
            except:
                print(str(e))
                print(f'failed for {path}')
        else:
            print("Incorrect data for ", path)

    raw_data.to_csv(data_path+raw_data_filename)
    
else:
    df = pd.read_csv(data_path+os.listdir(data_path)[1], index_col=0)

    growth_cols = list(df.columns[df.columns.str.contains('growth')])
    numeric_cols = [
        col for col in df.columns if col not in categorical_cols+growth_cols+['date']+deleted_cols
    ]
    raw_data = pd.read_csv(data_path+raw_data_filename, index_col=0)

In [ ]:
[col for col in os.listdir(data_path) if 'AAPL' in col] 

In [ ]:
categorical_cols = [col for col in categorical_cols if col not in deleted_cols]

column_dict = {
    'growth': growth_cols, 
    'categorical': [col for col in categorical_cols if col[:2] != 'is'],
    'bool': [col for col in categorical_cols if col[:2] == 'is'],
    'moving_avgs': list(raw_data.columns[raw_data.columns.str.contains('_ema_')]),
    'targets': list(raw_data.columns[raw_data.columns.str.contains('next_year')]),
    'hash_encoded':  ['sector', 'country', 'industry', 'exchange'],
    'one_hot_encoded': ['currency'],
    'date_categoricals': ['year', 'years_since_public', 'date'],
    'numeric': numeric_cols
}

In [ ]:
for varname in categorical_cols:
    try:
        print(varname, len(raw_data[varname].value_counts()))
    except:
        pass

In [ ]:
all_data.columns[~all_data.columns.str.contains('next_year_revenue|next_year_freeCashFlow')]

In [ ]:
print(len(raw_data))
billion = 1000000000

all_data = raw_data[raw_data['currency'].isin(['USD', 'GBp', 'EUR', 'CAD'])]

big_tickers = (
    list(all_data[all_data['revenue'] > billion*1000]['symbol'].unique()) +
    list(all_data[all_data['freeCashFlow'] > billion*1000]['symbol'].unique())
)

for ticker in big_tickers:
    all_data = all_data[all_data['symbol'] != ticker]

all_data = all_data.reset_index(drop=True)
print(len(all_data))


#### TARGETS #####
for col in column_dict['targets']:
    all_data[col+'_bn_scale_tg'] =  all_data[col]/billion
    
    
all_data, log_maxes = math_func_scaling(
    all_data, 
    column_dict['targets'], 
    write_to_new_df=False,
    prefix='_log_scale_tg',
    math_func='log'
)

all_data, sqrt_maxes = math_func_scaling(
    all_data, 
    column_dict['targets'], 
    write_to_new_df=False,
    prefix='_sqrt_scale_tg',
    math_func='sqrt'
)

log_maxes.to_csv('data/log_maxes.csv')
sqrt_maxes.to_csv('data/sqrt_maxes.csv')

column_dict['bn_scaled_targets'] = \
    list(all_data.columns[all_data.columns.str.contains('_bn_scale_tg')])

column_dict['log_scaled_targets'] = \
    list(all_data.columns[all_data.columns.str.contains('_log_scale_tg')])

column_dict['sqrt_scaled_targets'] = \
    list(all_data.columns[all_data.columns.str.contains('_sqrt_scale_tg')])


##### INPUTS #####
for col in numeric_cols+column_dict['moving_avgs']:
    all_data[col+'_bn_scale_inputs'] =  all_data[col]/billion

all_data, _ = math_func_scaling(
    all_data, 
    numeric_cols+column_dict['moving_avgs'], 
    write_to_new_df=False,
    prefix='_log_scale_inputs',
    math_func='log'
)

all_data, _ = math_func_scaling(
    all_data, 
    numeric_cols+column_dict['moving_avgs'], 
    write_to_new_df=False,
    prefix='_sqrt_scale_inputs',
    math_func='sqrt'
)


all_data, _ = add_scaled_vars(
    all_data,
    ['year', 'years_since_public'],
    write_to_new_df=False,
    prefix='_scaled_years'
)

all_data = one_hot_encoding(
    all_data, 
    ['currency'], 
    concat=True
) 


all_data, hash_encoding_columns = hash_encode_gt_features(
    all_data, 
    column_dict['hash_encoded'], 
    concat=True, 
    num_encode_dimensions=16, 
    hashing_method='md5'
)


column_dict['bn_scaled_inputs'] = \
    list(all_data.columns[all_data.columns.str.contains('_bn_scale_inputs')])

column_dict['log_scaled_inputs'] = \
    list(all_data.columns[all_data.columns.str.contains('_log_scale_inputs')])

column_dict['sqrt_scaled_inputs'] = \
    list(all_data.columns[all_data.columns.str.contains('_sqrt_scale_inputs')])

column_dict['scaled_years'] = \
    list(all_data.columns[all_data.columns.str.contains('_scaled_years')])

column_dict['one_hot_encoded_columns'] = \
    list(all_data.columns[all_data.columns.str.contains('encoded')])

column_dict['hash_encoded_columns'] = hash_encoding_columns 


print(all_data.shape)

# mask out next year with no targets
next_year_mask = all_data['next_year_revenue'].isna()
next_year = all_data[next_year_mask].reset_index(drop=True)
all_data = all_data[~next_year_mask].reset_index(drop=True)

next_year.to_csv('data/next_year.csv')
all_data.to_csv('data/all.csv')

with open('data/column_dict.json', 'w') as ff:
    json.dump(column_dict, ff) 

In [ ]:
# NANs
for index, val in all_data.isna().sum().iteritems():
    if val:
        print(index, val)

In [ ]:
for key, val in column_dict.items():
    print(key)
    print('\n\t', '\n\t'.join(val), "\n\n")

In [ ]:

dt = all_data[all_data['symbol'] == 'AAPL']
fig = px.scatter(
    dt, 
    x="year", 
    y=[
        'next_year_freeCashFlow_bn_scale_tg', 
        'next_year_freeCashFlow_log_scale_tg',
        'freeCashFlow_bn_scale_inputs',
        'freeCashFlow_log_scale_inputs'
    ], 
    #color="species"
)
fig.show()

In [ ]:
all_data = pd.read_csv('data/all.csv')
next_year = pd.read_csv('data/next_year.csv')
column_dict = json.load(open('data/column_dict.json', 'r'))

In [ ]:
for n in all_data.columns[all_data.columns.str.contains('ema')]:
    print(n)

In [ ]:
print("Possible Sections: \n ", '\n  '.join(list(column_dict.keys())), "\n\n")

remove_nan = True
cat = True
just_free_cash_flow_target = True
remove_revenue_above_1bn = True

remove_ema = 10
reg_type = 'bn'

specific_columns = []
segments = ['growth', 'bool'] 


affix = reg_type+'_cat' if cat else reg_type
        

segments = segments + [
    'one_hot_encoded_columns',
    'hash_encoded_columns',
    'scaled_years', 
] if not cat else segments + ['categorical'] 


if reg_type == 'log':
    segments = segments + ['log_scaled_inputs'] 
    targets = column_dict['log_scaled_targets']
    
elif reg_type == 'sqrt':
    segments = segments + ['sqrt_scaled_inputs']
    targets = column_dict['sqrt_scaled_targets']
    
else:
    segments = segments + ['bn_scaled_inputs']
    targets = column_dict['bn_scaled_targets']
    

input_cols = []
for segment in segments:
    input_cols.extend(column_dict[segment])

input_cols.extend(specific_columns)

if remove_ema:
    affix += f'_NO{remove_ema}EMA'
    input_cols = [
        col for col in input_cols 
        if str(remove_ema)+'_ema' not in col
    ]
    

print(len(all_data))

if remove_nan:
    mod_data = copy.deepcopy(
        all_data[~all_data[input_cols+targets].isnull().any(axis=1)]
    ).reset_index(drop=True)
    
    mod_next_year = copy.deepcopy(
        next_year[~next_year[input_cols].isnull().any(axis=1)]
    ).reset_index(drop=True)

else:
    mod_data = copy.deepcopy(all_data)
    mod_next_year = copy.deepcopy(next_year)
    affix += '_nans'
    
if just_free_cash_flow_target:
    affix += "_JFCF"
    targets = [col for col in targets if 'freeCashFlow' in col]
    
if remove_revenue_above_1bn:
    mod_data = mod_data[mod_data['4_ema_revenue'] < 1000000000]
    mod_next_year = mod_next_year[mod_next_year['4_ema_revenue'] < 1000000000]
    affix += '_UP21bn'
    
print(len(mod_data))
print(affix, "\n", segments, "\n", targets)

target_years = [2006, 2012, 2018]

to_float_cols = [col for col in input_cols if col not in column_dict['categorical']]
mod_data[to_float_cols] = mod_data[to_float_cols].astype(np.float32)
mod_next_year[to_float_cols] = mod_next_year[to_float_cols].astype(np.float32)

datasets = {
    'train': mod_data[mod_data['year'].isin(
        [
            year for year in mod_data['year'].unique() 
             if year not in target_years+[2019, 2020]
        ]
    )],
    'test': mod_data[mod_data['year'].isin(target_years)],
    'val': mod_data[mod_data['year'] == 2020],
    'next_year': mod_next_year
}

for key, data in datasets.items():
    data[input_cols+['symbol']].to_csv(
        f'data/performance_forecasting/{key}_input_{affix}.csv'
    )
    data[targets+['symbol']].to_csv(
        f'data/performance_forecasting/{key}_output_{affix}.csv'
    )

In [ ]:
for key, ds in datasets.items():
    print(key, len(ds))